<a href="https://colab.research.google.com/github/crisbpadilla/CapstoneProject-1-M2M/blob/main/CapstoneProject_2_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning- compressive strenght prediction modeling
  In the present capston project, the same data used in the first project will be scaled and transform to set up the data, prepared the training and testing data and evaluate the different regression models that could be used to predict the compressive strength of the concrete mix.

  So, the dataset was obtain from kaggle:

In [59]:
import pandas as pd
import numpy as np

concrete=pd.read_csv('concrete.csv')

In the first capston project it was possible to identify that the main components that helps to improve the development and higher values of compressive strength were the SCM and the superplastice.
So due this, we will firtly checked the correlation matrix between the variables to have  a better idea of it and see if this could help to take desicions in the future models.

In [60]:
from bokeh.plotting import figure, output_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, LinearColorMapper,ColorBar

output_notebook()
material_corr=concrete.corr()

material_corr = material_corr.stack().reset_index()
material_corr.columns = ['x', 'y', 'value']

material_corr['value'] = material_corr['value'].round(2)

source = ColumnDataSource(material_corr)

# Define color mapper
color_mapper = LinearColorMapper(palette="Inferno256", low=-1, high=1)

# Create figure
p = figure(title="Correlation Heatmap", x_range=list(concrete.columns), y_range=list(concrete.columns),
           x_axis_location="above", width=600, height=600,
           tools="hover", tooltips="@x, @y: @value{0.2f}")

p.rect(x="x", y="y", width=1, height=1, source=source,
       fill_color={'field': 'value', 'transform': color_mapper}, line_color=None)

# Add text annotations
from bokeh.models import LabelSet
labels = LabelSet(x='x', y='y', text='value', source=source,text_align='center',
                  text_baseline='middle',text_font_size='8pt',text_color='white')
p.add_layout(labels)

# Add color bar
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

p.xaxis.major_label_orientation = 0.75

# Show plot
show(p)


Reviewing the heatmap it is possible to observe the inverse correlation between water an superplastic, variable that could be flattened as they are both lightly correlated ( superplastic is used to speed up the compressive strength gain and the reduction of water in the mix).
 So we will check the performance of the model, reducing the demionsionality of this model , taking water and superplastic into one unique component.

# Preprocessing data

In [61]:
print(concrete.head())
print(concrete.info())

   cement   slag    ash  water  superplastic  coarseagg  fineagg  age  \
0   141.3  212.0    0.0  203.5           0.0      971.8    748.5   28   
1   168.9   42.2  124.3  158.3          10.8     1080.8    796.2   14   
2   250.0    0.0   95.7  187.4           5.5      956.9    861.2   28   
3   266.0  114.0    0.0  228.0           0.0      932.0    670.0   28   
4   154.8  183.4    0.0  193.3           9.1     1047.4    696.7   28   

   strength  
0     29.89  
1     23.51  
2     29.22  
3     45.85  
4     18.29  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cement        1030 non-null   float64
 1   slag          1030 non-null   float64
 2   ash           1030 non-null   float64
 3   water         1030 non-null   float64
 4   superplastic  1030 non-null   float64
 5   coarseagg     1030 non-null   float64
 6   fineagg       1030 non

the dataframe is build by the raw materials that composed the concrete mix  (kg/m^3), age (days) and compressive strength of the sample.

with the idea of see the impact of adding   categorical values to the model, depending the type of mix, the next code includes the next categories.

    * all mix
    * slag
    * ash
    * superplastic
    * slag-ash
    * slag-superplastic
    * ash-superplastic.
    * just cement


In [62]:
concrete_pluscat=concrete.copy()
# cuales son los escenarios? 9 (matriz 3x3) , la mitad menos 1 ~ 3
#a) slag-ash b)slag-superplastic c)ash-superplastic

concrete_pluscat['type_mix'] = 'all mix'
concrete_pluscat.loc[
    (concrete_pluscat['slag']>0) & (concrete_pluscat[['superplastic','ash']] == 0).all(axis=1),
    'type_mix',
] = 'slag'
concrete_pluscat.loc[
    (concrete_pluscat['ash']>0) & (concrete_pluscat[['superplastic','slag']] == 0).all(axis=1),
    'type_mix',
] = 'ash'
concrete_pluscat.loc[
    (concrete_pluscat['superplastic']>0) & (concrete_pluscat[['slag','ash']] == 0).all(axis=1),
    'type_mix',
] = 'superplastic'

concrete_pluscat.loc[
    (concrete_pluscat[['slag', 'ash']].gt(0).all(axis=1)) & (concrete_pluscat['superplastic'] == 0),
    'type_mix',
] = 'slag-ash'

concrete_pluscat.loc[
    (concrete_pluscat[['slag', 'superplastic']].gt(0).all(axis=1)) & (concrete_pluscat['ash'] == 0),
    'type_mix',
] = 'slag-superplastic'

concrete_pluscat.loc[
    (concrete_pluscat[['ash', 'superplastic']].gt(0).all(axis=1)) & (concrete_pluscat['slag'] == 0),
    'type_mix',
] = 'ash-superplastic'
concrete_pluscat.loc[
    (concrete_pluscat['cement']>0) & (concrete_pluscat[['superplastic','slag','ash']] == 0).all(axis=1),
    'type_mix',
] = 'just cement'

The dummies for the categorical values were created so we can added to the model later

In [63]:
# Create the dataframe with all the numerical and categorical values
concrete_data=pd.get_dummies(data=concrete_pluscat,columns=['type_mix'],drop_first=True).astype(int)
concrete_data.head()


,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength,type_mix_ash,type_mix_ash-superplastic,type_mix_just cement,type_mix_slag,type_mix_slag-superplastic,type_mix_superplastic
0,141,212,0,203,0,971,748,28,29,0,0,0,1,0,0
1,168,42,124,158,10,1080,796,14,23,0,0,0,0,0,0
2,250,0,95,187,5,956,861,28,29,0,1,0,0,0,0
3,266,114,0,228,0,932,670,28,45,0,0,0,1,0,0
4,154,183,0,193,9,1047,696,28,18,0,0,0,0,1,0


In [64]:
RED = "\033[31m"
RESET = "\033[0m"

ptg_data28=(len(concrete[concrete['age']<=100])/len(concrete))*100


print(f'The percentage of data under 100 days over the total is {RED}{round(ptg_data28,3)} %{RESET}')

print(f"{RED}{len(concrete[concrete['age']>100])}{RESET} samples are over 100 days")
print(f"{RED}{len(concrete[concrete['age']<=100])}{RESET} samples are under 100 days")

The percentage of data under 100 days over the total is 93.981 %
62 samples are over 100 days
968 samples are under 100 days


in the first capstone project, it is possible to identify that big amount of the data is below 100 days, using mathematical operations we can observe that ***93,981 %*** of the samples are obtain before the 100 days, so this data is the most representative of the whole dataset, being the samples above that age potential noise in the model. Cause of this, the samples above 100 days will be removed of the model an this will predict compressive strength between 0 and 100 days.

In [65]:
concrete_model_data=concrete_data[concrete_data['age']<=100]

In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X=concrete_model_data.drop(columns=['strength'])
y=concrete_model_data['strength']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)


In [67]:
# Seprating my numeric variables and nominal/categorical values
num_col=[a for a in concrete.columns]
num_col.remove('strength')
#numerical values
X_train_num=X_train[num_col]
X_test_num=X_test[num_col]
#nominal values
print(num_col)
X_train_nom=X_train.drop(columns=num_col)
X_test_nom=X_test.drop(columns=num_col)
print(X_train_nom.columns)

['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg', 'age']
Index(['type_mix_ash', 'type_mix_ash-superplastic', 'type_mix_just cement',
       'type_mix_slag', 'type_mix_slag-superplastic', 'type_mix_superplastic'],
      dtype='object')


##Scalling the data

In [68]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scale_data=StandardScaler()
print(pd.DataFrame(scale_data.fit_transform(X_train_num)))

X_train_num_scale=scale_data.fit_transform(X_train_num)
X_test_num_scale=scale_data.fit_transform(X_test_num)


            0         1         2         3         4         5         6  \
0   -0.637465 -0.879556  1.827661 -1.204233  0.814311  1.004786 -0.009859   
1   -0.397846 -0.879556  0.583704  0.318891  0.123870 -0.296336  0.930565   
2   -0.273245 -0.879556  0.956891  0.414086 -0.048740  0.701612 -0.244965   
3   -0.972931  0.622522  0.956891 -0.014292 -0.566570  0.436335 -0.388641   
4    1.394502  0.354709 -0.877945 -1.251830  1.677361 -1.521665  1.453023   
..        ...       ...       ...       ...       ...       ...       ...   
721  0.551044  1.321162 -0.877945 -0.252280  0.469091 -0.359497 -0.271088   
722 -0.263660 -0.879556  0.645902 -1.585013  1.332141  0.183690  1.479146   
723  1.595782 -0.600100  0.350462 -0.823452  0.814311 -0.068955 -0.832730   
724  2.352977 -0.879556 -0.877945  0.461684 -1.084401  1.926941 -2.125813   
725 -1.346737  1.554043 -0.877945  0.604476 -1.084401  0.941625  0.395046   

            7  
0   -1.031333  
1   -0.640509  
2   -0.143097  
3   -0.6405

###Lineal model

In [84]:
from sklearn import linear_model
lineal=linear_model.LinearRegression()
lineal.fit(X_train_num_scale,y_train)
print(lineal.score(X_train_num_scale,y_train))
print(lineal.score(X_test_num_scale,y_test))

0.7473723269003775
0.7961164636884345


###Reciprocal logaritmic


In [81]:
import numpy as np

def addReciprocalLogFeatures(numeric):
    log_feats = numeric.copy()
    valid = (log_feats != 1) & (log_feats > 0)
    log_feats[valid] = np.log(log_feats[valid]) / np.log(10)
    log_feats[log_feats <= 0] = 1e-10
    rec_log_feats = log_feats
    return np.hstack([numeric, rec_log_feats, numeric * rec_log_feats])

X_train_reclog = addReciprocalLogFeatures(X_train_num_scale)
X_test_reclog = addReciprocalLogFeatures(X_test_num_scale)


In [82]:
#log regression
lineal_rl=linear_model.LinearRegression()
lineal_rl.fit(X_train_reclog,y_train)
print(lineal_rl.score(X_train_reclog,y_train))
print(lineal_rl.score(X_test_reclog,y_test))

0.8358093804229129
0.8586121441362118


Linear and reciprocal logaritmic model, shown higher score in the testing samples than the training samples. which is not a good signal ....

###cubic model

In [89]:
#lineal regession gives a r2 too low and the function seems to fit to a log function
from sklearn.preprocessing import PolynomialFeatures,FunctionTransformer

from sklearn.pipeline import make_pipeline

square = PolynomialFeatures(degree=3, include_bias=False)
X_train_scale_square = square.fit_transform(X_train_num_scale)
X_test_scale_square = square.fit_transform(X_test_num_scale)

reg=linear_model.LinearRegression()
reg.fit(X_train_scale_square,y_train)
prediction_train=reg.predict(X_train_scale_square)
prediction_test=reg.predict(X_test_scale_square)

print(reg.score(X_train_scale_square,y_train))
print(reg.score(X_test_scale_square,y_test))


0.9475278877178278
0.8240061102703925


**Removing the data above 100 days improve the model consideraby  the scores in all 3 different models but just the cubic polynomial mode have  good score with a coherent output in both scores.**

In [85]:

X_train_ln_full = np.hstack([X_train_scale_square, X_train_nom])
X_test_ln_full = np.hstack([X_test_scale_square, X_test_nom])

X_train_ln_full.shape

(726, 170)

In [86]:
ln_model= linear_model.RidgeCV()
ln_model.fit(X_train_ln_full,y_train)
train_predict_ln=ln_model.predict(X_train_ln_full)
test_predict_ln=ln_model.predict(X_test_ln_full)
print(f'For the train data,the R^2 for the cuadratic model is : {round(ln_model.score(X_train_ln_full,y_train),3)}')
print(f'For the test data,the R^2 for the cuadratic model is : {round(ln_model.score(X_test_ln_full,y_test),3)}')

For the train data,the R^2 for the cuadratic model is : 0.942
For the test data,the R^2 for the cuadratic model is : 0.85


In [87]:
df0 = pd.DataFrame({"Predicted": test_predict_ln, "Actual": y_test})
df0['% Difference'] = (abs(df0['Predicted']-df0['Actual'])/df0['Actual'])*100

print("Percentage Difference between Predicted and Actual Values (cubic model)")
print(df0.head())
print("\nMean % Difference between Predicted and Actual Values: " + str(round(df0['% Difference'].mean(),3)) +"%")

Percentage Difference between Predicted and Actual Values (cubic model)
     Predicted  Actual  % Difference
894  45.492201      45      1.093780
389  32.987947      32      3.087335
279  69.295462      66      4.993125
40   22.888452      24      4.631449
562  39.123416      42      6.849009

Mean % Difference between Predicted and Actual Values: 15.742%


## applying PCA

In [76]:
X_train_WS=X_train[['water','superplastic']]
X_train_WS_scaled= scale_data.fit_transform(X_train_WS)
X_test_WS=X_test[['water','superplastic']]
X_test_WS_scaled= scale_data.fit_transform(X_test_WS)


pca=PCA(n_components=1)
X_train_pca=pca.fit_transform(X_train_WS_scaled)
X_test_pca=pca.fit_transform(X_test_WS_scaled)
# train and test data without water and superplastic

col=[a for a in concrete.columns]
items_to_remove = ['strength','water','superplastic']
col = [c for c in col if c not in items_to_remove]

X_train_non_PCA=X_train[col]
X_test_non_PCA=X_test[col]
#scaled data
X_train_non_PCA_scale=scale_data.fit_transform(X_train_non_PCA)
X_test_non_PCA_scale=scale_data.fit_transform(X_test_non_PCA)

X_train_PCA_scaled=np.hstack([X_train_pca,X_train_non_PCA_scale])
X_test_PCA_scaled=np.hstack([X_test_pca,X_test_non_PCA_scale])


In [90]:
cubic = PolynomialFeatures(degree=2, include_bias=False)
X_train_PCA_cubic = cubic.fit_transform(X_train_PCA_scaled)
X_test_PCA_cubic = cubic.fit_transform(X_test_PCA_scaled)

reg=linear_model.LinearRegression()
reg.fit(X_train_PCA_cubic,y_train)
prediction_train=reg.predict(X_train_PCA_cubic)
prediction_test=reg.predict(X_test_PCA_cubic)

print(reg.score(X_train_PCA_cubic,y_train))
print(reg.score(X_test_PCA_cubic,y_test))

0.8630888954701526
0.8844564171099878


In this case the R^2 for the train data was reduced but the R^2 for the test data increased in relation with the previous model.
 So, now left to add the nominal variabes to verify the score with this model.

In [91]:

X_train_ln_full = np.hstack([X_train_PCA_cubic, X_train_nom])
X_test_ln_full = np.hstack([X_test_PCA_cubic, X_test_nom])


In [92]:
ln_model= linear_model.RidgeCV()
ln_model.fit(X_train_ln_full,y_train)
train_predict_pca=ln_model.predict(X_train_ln_full)
test_predict_pca=ln_model.predict(X_test_ln_full)
print(ln_model.score(X_train_ln_full,y_train))
print(ln_model.score(X_test_ln_full,y_test))

0.8675694350629071
0.8874690699590595


In [93]:
df = pd.DataFrame({"Predicted": test_predict_pca, "Actual": y_test})
df['% Difference'] = (abs(df['Predicted']-df['Actual'])/df['Actual'])*100

print("Percentage Difference between Predicted and Actual Values (cubic model)")
print(df.head())
print("\nMean % Difference between Predicted and Actual Values: " + str(round(df['% Difference'].mean(),3)) +"%")

Percentage Difference between Predicted and Actual Values (cubic model)
     Predicted  Actual  % Difference
894  48.384165      45      7.520368
389  31.790910      32      0.653405
279  62.451535      66      5.376462
40   20.153147      24     16.028554
562  36.152423      42     13.922803

Mean % Difference between Predicted and Actual Values: 15.763%
